In [1]:
import os
import time
import inspect
import math
import pickle
from tqdm import tqdm
from contextlib import nullcontext
import numpy as np
import torch
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
from torch.nn import Linear as nnLinear
import torch.nn as nn
import torch.nn.functional as F

from mamba_ssm.models.config_mamba import MambaConfig
from models import *
from models.ttt import TTTLinear, TTTConfig, TTTForCausalLM
from models.PreCo import PreCoNewConfig, PreCoNewModel
from models.preco_nogain import PreCoNoGainConfig, PreCoNoGainModel
# import wandb

/root/miniconda3/envs/longhorn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
master_seed = 1337
out_dir = 'results/slim_results'
# 🚀 與 run.sh 實際訓練參數一致
eval_interval = 200     # 評估間隔 (run.sh: --eval_interval=200)
log_interval = 10       # 日誌間隔
eval_iters = 50         # 評估迭代次數
eval_only = False
always_save_checkpoint = False

In [3]:
model_name = "ttt" # llama, gla, rwkv, retnet, mamba, longhorn, ttt, preco

In [4]:
dataset = 'Slim'
do_eval = True
gradient_accumulation_steps = 4   # 實際梯度累積步數 (run.sh: grad_accum=4)
# 🚀 與 run.sh 實際訓練參數一致
batch_size = 4          # 實際批次大小 (run.sh: batch_size=4)
block_size = 2048       # 序列長度 (run.sh: block_size=2048)
use_branch_losses = False  # 僅用 CE 端到端訓練兩分支（建議）
branch_longhorn_weight = 1.0
branch_ttt_weight = 1.0

### 工具函數

In [5]:
def setup_distributed():
    """設置分布式訓練環境（單GPU優化版）"""
    ddp = int(os.environ.get('RANK', -1)) != -1
    if ddp:
        # 多GPU分布式訓練設置
        init_process_group(backend=backend)
        ddp_rank = int(os.environ['RANK'])
        ddp_local_rank = int(os.environ['LOCAL_RANK'])
        ddp_world_size = int(os.environ['WORLD_SIZE'])
        device_name = f'cuda:{ddp_local_rank}'
        torch.cuda.set_device(device_name)
        master_process = ddp_rank == 0
        seed_offset = ddp_rank
        return ddp, device_name, master_process, seed_offset, ddp_world_size, ddp_local_rank
    else:
        # 單GPU訓練設置（您的情況）
        print("檢測到單GPU環境，使用標準訓練模式")
        return False, device, True, 0, 1, 0


In [6]:
def load_data(dataset_name, block_size):
    """載入訓練數據"""
    data_dir = os.path.join('data', dataset_name)
    
    print(f"使用 PyTorch 原生數據載入方式")
    
    # 載入數據
    train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
    val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
    
    print(f" 載入數據:")
    print(f" Train tokens: {len(train_data):,}")
    print(f" Val tokens: {len(val_data):,}")
    print(f" 序列長度: {block_size}")
    
    # 載入詞彙表信息
    meta_path = os.path.join(data_dir, 'meta.pkl')
    if os.path.exists(meta_path):
        with open(meta_path, 'rb') as f:
            meta = pickle.load(f)
        vocab_size = meta.get('vocab_size', 50257)
        print(f"載入meta資訊: vocab_size={vocab_size}")
    else:
        vocab_size = 50257
        print(f"未找到meta.pkl，使用預設vocab_size={vocab_size}")
    
    return train_data, val_data, vocab_size

In [7]:
def get_batch_fn(train_data, val_data, batch_size, block_size, device_type, device):
    """創建批次獲取函數"""
    def get_batch(split):
        data = train_data if split == 'train' else val_data
        ix = torch.randint(len(data) - block_size, (batch_size,))
        x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
        y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
        
        if device_type == 'cuda':
            x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
        else:
            x, y = x.to(device), y.to(device)
        
        return x, y
    return get_batch

def calculate_ppl(loss):
    """計算困惑度"""
    if loss > 700:
        return float('inf')
    try:
        return math.exp(loss)
    except OverflowError:
        return float('inf')

### 學習率調度器

In [8]:
def get_lr(iter_num, warmup_iters, lr_decay_iters, learning_rate, min_lr, decay_lr=True):
    """主學習率調度器 - 餘弦衰減 + warmup"""
    if not decay_lr:
        return learning_rate
    
    # 線性 warmup
    if iter_num < warmup_iters:
        return learning_rate * iter_num / warmup_iters
    
    # 達到衰減終點：固定最小學習率
    if iter_num >= lr_decay_iters:
        return min_lr
    
    # 餘弦衰減
    decay_ratio = (iter_num - warmup_iters) / (lr_decay_iters - warmup_iters)
    decay_ratio = max(0.0, min(1.0, decay_ratio))
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (learning_rate - min_lr)

def get_ttt_lr_mult(iter_num, lr_decay_iters):
    """TTT 學習率倍數調度器（僅用於 preco_nogain 和 ttt）"""
    base_mult = 1.0
    warmup_steps = 500
    
    if iter_num < warmup_steps:
        # 餘弦 warmup: 0.1 → 1.0
        progress = iter_num / warmup_steps
        return 0.1 + 0.9 * (1.0 - math.cos(math.pi * progress)) / 2.0
    elif iter_num < 2000:
        return base_mult
    elif iter_num >= 3000:
        # 餘弦衰減: 1.0 → 0.3
        decay_start = 3000
        decay_ratio = (iter_num - decay_start) / (lr_decay_iters - decay_start)
        decay_ratio = min(decay_ratio, 1.0)
        min_ttt_mult = 0.3
        coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
        return min_ttt_mult + coeff * (base_mult - min_ttt_mult)
    else:
        return base_mult

### 模型創建函數

In [9]:
def create_estimate_loss_fn_corrected(model, model_name, get_batch, eval_iters, device_type):
    """🔧 修正後的損失評估函數 - 明確區分 total_loss 和 ce_loss"""
    
    @torch.no_grad()
    def estimate_loss(iter_num=0):
        out = {}
        model.eval()
        
        for split in ['train', 'val']:
            # 🎯 關鍵修正：明確區分 total_loss 和 ce_loss
            total_losses_list = []    # 用於報告 loss (total_loss)
            ce_losses_list = []       # 用於計算 perplexity (ce_loss)
            
            # PreCo 系列特定統計
            if model_name == "preco":
                longhorn_losses, ttt_losses = [], []
                kalman_means, kalman_stds = [], []
            
            bad_batches = 0
            
            for k in range(eval_iters):
                x, y = get_batch(split)
                
                # 評估時使用 FP32 以提高穩定性
                with torch.cuda.amp.autocast(enabled=False):
                    if model_name == "ttt":
                        outputs = model(input_ids=x, labels=y)
                        if isinstance(outputs, tuple):
                            logits, loss = outputs[0], outputs[1]
                        else:
                            logits, loss = outputs.logits, outputs.loss
                        
                        loss_val = float(loss.item()) if hasattr(loss, 'item') else float(loss)
                        if math.isfinite(loss_val):
                            # TTT 模型：loss 就是 ce_loss
                            total_losses_list.append(loss_val)
                            ce_losses_list.append(loss_val)
                        else:
                            bad_batches += 1
                            
                    elif model_name == "preco":
                        # PreCo 使用 TTT 原生自適應學習率
                        logits, loss_dict = model(x, y, compute_branch_loss=False)
                        
                        # 轉換所有損失為 float
                        for key in list(loss_dict.keys()):
                            if hasattr(loss_dict[key], 'item'):
                                loss_dict[key] = loss_dict[key].item()
                            else:
                                loss_dict[key] = float(loss_dict[key])
                        
                        total_loss_val = float(loss_dict.get('total_loss', float('nan')))
                        ce_loss_val = float(loss_dict.get('ce_loss', float('nan')))
                        
                        if math.isfinite(total_loss_val) and math.isfinite(ce_loss_val):
                            # 🎯 關鍵修正：明確區分用途
                            total_losses_list.append(total_loss_val)  # 報告 total_loss
                            ce_losses_list.append(ce_loss_val)        # 計算 perplexity
                            
                            longhorn_losses.append(float(loss_dict.get('longhorn_loss', 0.0)))
                            ttt_losses.append(float(loss_dict.get('ttt_loss', 0.0)))
                            kalman_means.append(float(loss_dict.get('kalman_mean', 0.0)))
                            kalman_stds.append(float(loss_dict.get('kalman_std', 0.0)))
                        else:
                            bad_batches += 1
                            
                    else:
                        # 其他模型 (longhorn, etc.)
                        logits, loss = model(x, y)
                        loss_val = float(loss.item()) if hasattr(loss, 'item') else float(loss)
                        if math.isfinite(loss_val):
                            # 其他模型：loss 就是 ce_loss
                            total_losses_list.append(loss_val)
                            ce_losses_list.append(loss_val)
                        else:
                            bad_batches += 1
            
            # 🎯 計算平均損失：明確區分用途
            mean_total_loss = (sum(total_losses_list) / len(total_losses_list)) if len(total_losses_list) > 0 else float('inf')
            mean_ce_loss = (sum(ce_losses_list) / len(ce_losses_list)) if len(ce_losses_list) > 0 else float('inf')
            
            result = {
                'loss': float(mean_total_loss),           # 報告 total_loss
                'ppl': calculate_ppl(float(mean_ce_loss)) # 基於 ce_loss 計算 perplexity ✅
            }
            
            # 添加 PreCo 統計
            if model_name == "preco":
                result.update({
                    'ce_loss': float(mean_ce_loss),  # 明確報告 ce_loss
                    'longhorn_loss': float(sum(longhorn_losses)/len(longhorn_losses)) if len(longhorn_losses)>0 else float('inf'),
                    'ttt_loss': float(sum(ttt_losses)/len(ttt_losses)) if len(ttt_losses)>0 else float('inf'),
                    'kalman_mean': float(sum(kalman_means)/len(kalman_means)) if len(kalman_means)>0 else 0.0,
                    'kalman_std': float(sum(kalman_stds)/len(kalman_stds)) if len(kalman_stds)>0 else 0.0,
                    'dropped_batches': bad_batches,
                })
            
            out[split] = result
        
        model.train()
        return out
    
    return estimate_loss

print("  - loss 報告: total_loss")
print("  - ppl 計算: exp(ce_loss)")

  - loss 報告: total_loss
  - ppl 計算: exp(ce_loss)


### 修正說明和使用示例


In [10]:
# # 📊 修正說明：Loss vs Perplexity 的正確計算

# print("🔧 問題分析:")
# print("  原始代碼問題：")
# print("    - losses_list 存儲的是 total_loss (可能包含 longhorn/ttt loss)")
# print("    - 但用 total_loss 計算 perplexity 是錯誤的")
# print("    - perplexity 應該基於 ce_loss 計算")
# print()

# print("✅ 修正方案:")
# print("  明確區分兩個用途：")
# print("    - loss 報告: total_loss (完整的訓練損失)")
# print("    - ppl 計算: exp(ce_loss) (語言模型的困惑度)")
# print()

# print("📈 不同模型的處理:")
# print("  TTT/Longhorn 模型:")
# print("    - loss = ce_loss (單純的交叉熵)")
# print("    - ppl = exp(ce_loss)")
# print()
# print("  PreCo 模型:")
# print("    - loss = total_loss (可能包含分支損失)")
# print("    - ppl = exp(ce_loss) (僅基於交叉熵)")
# print("    - 額外報告: ce_loss, longhorn_loss, ttt_loss")
# print()

# # 使用示例
# print("🚀 使用修正後的評估函數:")
# print("# 替換原來的評估函數")
# print("estimate_loss_corrected = create_estimate_loss_fn_corrected(model, model_name, get_batch, eval_iters, device_type)")
# print()
# print("# 評估結果示例 (PreCo 模型):")
# print("losses = estimate_loss_corrected()")
# print("print(f'Loss: {losses[\"train\"][\"loss\"]:.4f}')      # total_loss")
# print("print(f'PPL: {losses[\"train\"][\"ppl\"]:.2f}')        # exp(ce_loss)")
# print("print(f'CE Loss: {losses[\"train\"][\"ce_loss\"]:.4f}') # ce_loss")


In [11]:
# 🚀 修正後的模型創建 - 與實際訓練參數一致

from models.longhorn import LonghornLM, LonghornConfig
from models.ttt import TTTLinear, TTTConfig, TTTForCausalLM
from models.PreCo import PreCoNewConfig, PreCoNewModel

# ========================================
# 1. Longhorn 模型 (與 run.sh 和 train.py 一致)
# ========================================
print("🚀 創建 Longhorn 模型 - 實際訓練配置")

Longhorn_model = LonghornLM(
    LonghornConfig(
        vocab_size=50257,   # 實際詞彙表大小 (與 train.py 一致)
        d_model=768,        # 模型維度 (run.sh: --n_embd=768)
        n_layer=12,         # 層數 (run.sh: --n_layer=12)
        ssm_cfg={
            'd_state': 16,  # SSM 狀態維度 (train.py 實際配置)
            'd_conv': 4,    # 卷積核大小 (train.py 實際配置)
            'expand': 4     # 內部維度擴展倍數 (train.py: 4×768=3072)
        },
        # 其他配置與 train.py 一致
        rms_norm=True,
        residual_in_fp32=True,
        fused_add_norm=True,
        tie_embeddings=True
    )
)

print(f"Longhorn 模型參數量: {sum(p.numel() for p in Longhorn_model.parameters())/1e6:.1f}M")

# ========================================
# 2. TTT 模型 (與 run.sh 一致)
# ========================================
print("🚀 創建 TTT 模型 - 125M 配置")

TTT_model = TTTForCausalLM(
    TTTConfig(
        vocab_size=50257,                    # 實際詞彙表大小
        hidden_size=768,                     # 隱藏層維度 (run.sh: --n_embd=768)
        intermediate_size=2048,              # MLP 中間層維度 (768*2.67≈2048)
        num_hidden_layers=12,                # Transformer 層數 (run.sh: --n_layer=12)
        num_attention_heads=12,              # 注意力頭數 (run.sh: --n_head=12)
        max_position_embeddings=2048,        # 最大位置編碼 (run.sh: --block_size=2048)
        
        # TTT 特定參數 (與 run.sh 一致)
        ttt_base_lr=1.0,                     # TTT 基礎學習率
        mini_batch_size=16,                  # 小批次大小
        use_gate=False,                      # 門控機制 (run.sh: --use_gate=False)
        share_qk=False,                      # Q/K 共享 (run.sh: --share_qk=False)
        ttt_layer_type="linear",             # TTT 層類型
        pre_conv=True,                       # 預卷積 (run.sh: --pre_conv=True)
        conv_kernel=4,                       # 卷積核大小 (run.sh: --conv_kernel=4)
        scan_checkpoint_group_size=0,        # 梯度檢查點
        
        # 標準參數
        dropout=0.0,                         # Dropout (run.sh: --dropout=0.0)
        pad_token_id=0,
        bos_token_id=2,
        eos_token_id=3,
        tie_word_embeddings=True,
    )
)

print(f"TTT 模型參數量: {sum(p.numel() for p in TTT_model.parameters())/1e6:.1f}M")

# ========================================
# 3. PreCo 模型 (與 run.sh 一致)
# ========================================
print("創建 PreCo 模型 - 127M 配置")

PreCo_model = PreCoNewModel(
    PreCoNewConfig(
        vocab_size=50257,                    # 實際詞彙表大小
        d_model=512,                         # 模型維度 (run.sh: --longhorn_d_model=512)
        n_layer=12,                          # 層數 (run.sh: --longhorn_n_layer=12)
        
        # Longhorn 配置 (與 run.sh 一致)
        d_state=8,                           # SSM 狀態維度 (run.sh: --longhorn_d_state=8)
        d_conv=3,                            # 卷積核大小 (固定為3)
        expand=6,                            # SSM 擴展倍數 (run.sh: --longhorn_ssm_expand=6)
        
        # TTT 配置 (與 run.sh 一致)
        ttt_num_heads=8,                     # TTT 頭數 (run.sh: --ttt_num_heads=8)
        ttt_num_layers=1,                    # TTT 層數 (run.sh: --ttt_num_layers=1)
        mini_batch_size=16,                  # 小批次大小 (run.sh: --mini_batch_size=16)
        
        # 其他參數
        dropout=0.1,                         # Dropout (run.sh: --dropout=0.1)
    )
)


🚀 創建 Longhorn 模型 - 實際訓練配置
Longhorn 模型參數量: 128.5M
🚀 創建 TTT 模型 - 125M 配置
TTT 模型參數量: 124.4M
創建 PreCo 模型 - 127M 配置


In [12]:
print(TTT_model)

TTTForCausalLM(
  (model): TTTModel(
    (embed_tokens): Embedding(50257, 768, padding_idx=0)
    (layers): ModuleList(
      (0-11): 12 x Block(
        (seq_modeling_block): TTTLinear(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=768, bias=False)
          (v_proj): Linear(in_features=768, out_features=768, bias=False)
          (o_proj): Linear(in_features=768, out_features=768, bias=False)
          (rotary_emb): RotaryEmbedding()
          (post_norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        )
        (mlp): SwiGluMLP(
          (gate_proj): Linear(in_features=768, out_features=2048, bias=False)
          (up_proj): Linear(in_features=768, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=768, bias=False)
          (act_fn): SiLU()
        )
        (conv): Conv(
          (norm): RMSNorm()
          (conv): Conv1d(768, 768, kernel_s

### 評估函數

In [13]:
def create_estimate_loss_fn(model, model_name, get_batch, eval_iters, device_type):
    """創建損失評估函數"""
    
    @torch.no_grad()
    def estimate_loss(iter_num=0):
        out = {}
        model.eval()
        
        for split in ['train', 'val']:
            losses_list = []
            ppl_losses_list = []
            
            # PreCo 系列特定統計
            if model_name == "preco":
                ce_losses, longhorn_losses, ttt_losses = [], [], []
                kalman_means, kalman_stds = [], []
            
            bad_batches = 0
            
            for k in range(eval_iters):
                x, y = get_batch(split)
                
                # 評估時使用 FP32 以提高穩定性
                with torch.cuda.amp.autocast(enabled=False):
                    if model_name == "ttt":
                        outputs = model(input_ids=x, labels=y)
                        if isinstance(outputs, tuple):
                            logits, loss = outputs[0], outputs[1]
                        else:
                            logits, loss = outputs.logits, outputs.loss
                        
                        loss_val = float(loss.item()) if hasattr(loss, 'item') else float(loss)
                        if math.isfinite(loss_val):
                            losses_list.append(loss_val)
                            ppl_losses_list.append(loss_val)
                        else:
                            bad_batches += 1
                            
                    elif model_name == "preco":
                        # PreCo 使用 TTT 原生自適應學習率
                        logits, loss_dict = model(x, y, compute_branch_loss=False)
                        
                        # 轉換所有損失為 float
                        for key in list(loss_dict.keys()):
                            if hasattr(loss_dict[key], 'item'):
                                loss_dict[key] = loss_dict[key].item()
                            else:
                                loss_dict[key] = float(loss_dict[key])
                        
                        total_loss_val = float(loss_dict.get('total_loss', float('nan')))
                        ce_loss_val = float(loss_dict.get('ce_loss', float('nan')))
                        
                        if math.isfinite(total_loss_val) and math.isfinite(ce_loss_val):
                            losses_list.append(total_loss_val)
                            ppl_losses_list.append(ce_loss_val)
                            
                            ce_losses.append(ce_loss_val)
                            longhorn_losses.append(float(loss_dict.get('longhorn_loss', 0.0)))
                            ttt_losses.append(float(loss_dict.get('ttt_loss', 0.0)))
                            kalman_means.append(float(loss_dict.get('kalman_mean', 0.0)))
                            kalman_stds.append(float(loss_dict.get('kalman_std', 0.0)))
                        else:
                            bad_batches += 1
                            
                    else:
                        logits, loss = model(x, y)
                        loss_val = float(loss.item()) if hasattr(loss, 'item') else float(loss)
                        if math.isfinite(loss_val):
                            losses_list.append(loss_val)
                            ppl_losses_list.append(loss_val)
                        else:
                            bad_batches += 1
            
            # 計算平均損失
            mean_loss = (sum(losses_list) / len(losses_list)) if len(losses_list) > 0 else float('inf')
            mean_ppl_loss = (sum(ppl_losses_list) / len(ppl_losses_list)) if len(ppl_losses_list) > 0 else float('inf')
            
            result = {
                'loss': float(mean_loss),
                'ppl': calculate_ppl(float(mean_ppl_loss)),
                'dropped_batches': bad_batches
            }
            
            # 添加 PreCo 統計
            if model_name == "preco":
                result.update({
                    'ce_loss': float(sum(ce_losses)/len(ce_losses)) if len(ce_losses)>0 else float('inf'),
                    'longhorn_loss': float(sum(longhorn_losses)/len(longhorn_losses)) if len(longhorn_losses)>0 else float('inf'),
                    'ttt_loss': float(sum(ttt_losses)/len(ttt_losses)) if len(ttt_losses)>0 else float('inf'),
                    'kalman_mean': float(sum(kalman_means)/len(kalman_means)) if len(kalman_means)>0 else 0.0,
                    'kalman_std': float(sum(kalman_stds)/len(kalman_stds)) if len(kalman_stds)>0 else 0.0,
                })
            
            out[split] = result
        
        model.train()
        return out
    
    return estimate_loss

### 主訓練函數

In [14]:
# 🚀 訓練循環函數

def train_loop(model, optimizer, scaler, ctx, estimate_loss, tokens_per_iter, use_wandb=False):
    """主訓練循環"""
    
    # 訓練參數
    learning_rate = 1e-3
    max_iters = 9600
    warmup_iters = 960
    lr_decay_iters = max_iters
    min_lr = 1e-5
    grad_clip = 1.0
    
    # 統計變量
    iter_num = 0
    best_val_loss = 1e9
    t0 = time.time()
    
    # 預取第一個批次
    x, y = get_batch('train')
    
    print("開始訓練循環...")
    print(f"總迭代次數: {max_iters}")
    print(f"評估間隔: {eval_interval}")
    print(f"每次迭代處理 tokens: {tokens_per_iter:,}")
    
    # ========================================
    # 主訓練循環
    # ========================================
    while True:
        
        # ========================================
        # 1. 設置學習率
        # ========================================
        lr = get_lr(iter_num, warmup_iters, lr_decay_iters, learning_rate, min_lr, decay_lr=True)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        
        # ========================================
        # 2. 評估和檢查點保存
        # ========================================
        if iter_num % eval_interval == 0:
            losses = estimate_loss(iter_num)
            train_loss = losses['train']['loss']
            val_loss = losses['val']['loss']
            
            print(f"step {iter_num}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")
            print(f"train ppl {losses['train']['ppl']:.2f}, val ppl {losses['val']['ppl']:.2f}")
            
            # Train/Val gap 監控
            gap = val_loss - train_loss
            gap_ratio = gap / train_loss if train_loss > 0 else 0
            print(f"📈 Train/Val Gap: {gap:.4f} ({gap_ratio*100:.2f}%)")
            
            # PreCo 詳細統計
            if model_name == "preco":
                print(f"  📊 PRECO 詳細統計:")
                print(f"    CE Loss: {losses['train']['ce_loss']:.4f} / {losses['val']['ce_loss']:.4f}")
                print(f"    Kalman 統計: mean={losses['train']['kalman_mean']:.4f}, std={losses['train']['kalman_std']:.4f}")
            
            # wandb 記錄
            if use_wandb:
                log_data = {
                    "train/loss": train_loss, "val/loss": val_loss,
                    "train/ppl": losses['train']['ppl'], "val/ppl": losses['val']['ppl'],
                    "learning_rate": lr, "iter": iter_num,
                }
                
                if model_name == "preco":
                    log_data.update({
                        "preco/adaptive_ttt_lr": "native",
                        "preco/kalman_mean": losses['train']['kalman_mean'],
                        "preco/kalman_std": losses['train']['kalman_std'],
                    })
                
                wandb.log(log_data)
            
            # 保存最佳檢查點
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                print(f"新的最佳驗證損失: {best_val_loss:.4f}")
        
        # ========================================
        # 3. 前向和反向傳播
        # ========================================
        for micro_step in range(gradient_accumulation_steps):
            with ctx:
                if model_name == "ttt":
                    outputs = model(input_ids=x, labels=y)
                    if isinstance(outputs, tuple):
                        logits, loss = outputs[0], outputs[1]
                    else:
                        logits, loss = outputs.logits, outputs.loss
                    loss = loss / gradient_accumulation_steps
                    
                elif model_name == "preco":
                    logits, loss_dict = model(x, y, compute_branch_loss=False)
                    loss = loss_dict['total_loss'] / gradient_accumulation_steps
                    
                else:
                    logits, loss = model(x, y)
                    loss = loss / gradient_accumulation_steps
            
            # 異步預取下一個批次
            x, y = get_batch('train')
            scaler.scale(loss).backward()
        
        # ========================================
        # 4. 梯度裁剪和優化器步驟
        # ========================================
        if grad_clip != 0.0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)
        
        # ========================================
        # 5. 日誌記錄
        # ========================================
        t1 = time.time()
        dt = t1 - t0
        t0 = t1
        
        if iter_num % (log_interval * 2) == 0:
            lossf = loss.item() * gradient_accumulation_steps
            print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
            
            if model_name == "preco":
                print(f"TTT 自適應學習率: 原生機制")
        
        iter_num += 1
        
        # ========================================
        # 6. 終止條件
        # ========================================
        if iter_num > max_iters:
            break
    
    # 清理
    if use_wandb:
        wandb.finish()
    
    print("🎉 訓練完成！")
    return model, best_val_loss

print("訓練循環函數已定義")


訓練循環函數已定義


### 開始訓練

In [15]:
# ========================================
# 1. 基本配置設置
# ========================================

# 訓練參數 (與 run.sh 一致)
learning_rate = 1e-3      # run.sh: lr=2.5e-3
weight_decay = 0.1          # run.sh: wd=0.1
max_iters = 9600            # run.sh: max_iters=9600
warmup_iters = 960          # run.sh: warmup=960
lr_decay_iters = max_iters  # run.sh: --lr_decay_iters=$max_iters
min_lr = 1e-5               # 最小學習率
beta1, beta2 = 0.9, 0.95    # Adam 參數
grad_clip = 1.0             # 梯度裁剪

# 設備和精度配置
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device_type = 'cuda' if 'cuda' in device else 'cpu'
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# print(f"🚀 訓練配置:")
# print(f"  - 設備: {device}")
# print(f"  - 精度: {dtype}")
# print(f"  - 學習率: {learning_rate}")
# print(f"  - 最大迭代: {max_iters}")
# print(f"  - 批次大小: {batch_size}")
# print(f"  - 序列長度: {block_size}")

# ========================================
# 2. 數據載入
# ========================================
train_data, val_data, vocab_size = load_data(dataset, block_size)
get_batch = get_batch_fn(train_data, val_data, batch_size, block_size, device_type, device)
# ========================================
# 3. 模型選擇和初始化
# ========================================
print(f"創建模型: {model_name}")

if model_name == "longhorn":
    model = Longhorn_model
elif model_name == "ttt":
    model = TTT_model
elif model_name == "preco":
    model = PreCo_model
else:
    raise ValueError(f"不支持的模型類型: {model_name}")

model.to(device)

# 僅 TTT 模型需要轉換精度
if model_name == "ttt":
    model = model.to(ptdtype)

# ========================================
# 4. 優化器設置
# ========================================
param_dict = {pn: p for pn, p in model.named_parameters() if p.requires_grad}
decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]

optim_groups = [
    {'params': decay_params, 'weight_decay': weight_decay},
    {'params': nodecay_params, 'weight_decay': 0.0}
]

total_params = sum(p.numel() for p in decay_params) + sum(p.numel() for p in nodecay_params)
print(f"模型參數總量: {total_params/1e6:.1f}M")

# 創建優化器
fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
use_fused = fused_available and device_type == 'cuda'
extra_args = dict(fused=True) if use_fused else dict()

optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, 
                             betas=(beta1, beta2), weight_decay=weight_decay, **extra_args)
print(f"使用融合 AdamW: {use_fused}")

# 設置梯度縮放器
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# ========================================
# 5. 初始化 wandb (可選)
# ========================================
experiment_name = f"{dataset}_{block_size * batch_size * gradient_accumulation_steps * max_iters / 1e9:.1f}_block{block_size}_{model_name}"

wandb_config = {
    "model_name": model_name, "dataset": dataset, "block_size": block_size,
    "batch_size": batch_size, "gradient_accumulation_steps": gradient_accumulation_steps,
    "learning_rate": learning_rate, "max_iters": max_iters,
    "vocab_size": vocab_size, "total_params": total_params,
}

# 添加模型特定配置
if model_name == "preco":
    wandb_config.update({
        "longhorn_d_model": 512,
        "ttt_num_heads": 8,
        "training_objective": "end_to_end_cross_entropy",
    })

try:
    wandb.init(project="2048", name=experiment_name, config=wandb_config)
    use_wandb = True
    print("wandb 初始化成功")
except:
    use_wandb = False
    print("wandb 初始化失敗，跳過在線記錄")

# ========================================
# 6. 創建評估函數
# ========================================
estimate_loss = create_estimate_loss_fn(model, model_name, get_batch, eval_iters, device_type)

# 訓練統計
tokens_per_iter = gradient_accumulation_steps * batch_size * block_size
print(f"每次迭代處理 tokens: {tokens_per_iter:,}")

print("初始化完成，準備開始訓練！")


使用 PyTorch 原生數據載入方式
 載入數據:
 Train tokens: 943,635,020
 Val tokens: 9,101,108
 序列長度: 2048
載入meta資訊: vocab_size=50257
創建模型: ttt
模型參數總量: 124.4M
使用融合 AdamW: True
wandb 初始化失敗，跳過在線記錄
每次迭代處理 tokens: 32,768
初始化完成，準備開始訓練！


In [16]:
# 🚀 執行訓練循環

# 統計變量
iter_num = 0
best_val_loss = 1e9
t0 = time.time()

# 預取第一個批次
x, y = get_batch('train')

print("\\n🚀 開始訓練循環...")
print(f"總迭代次數: {max_iters}")
print(f"評估間隔: {eval_interval}")
print(f"每次迭代處理 tokens: {tokens_per_iter:,}")

# ========================================
# 主訓練循環
# ========================================
while True:
    
    # ========================================
    # 1. 設置學習率
    # ========================================
    lr = get_lr(iter_num, warmup_iters, lr_decay_iters, learning_rate, min_lr, decay_lr=True)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    
    # ========================================
    # 2. 評估和檢查點保存
    # ========================================
    if iter_num % eval_interval == 0:
        losses = estimate_loss(iter_num)
        train_loss = losses['train']['loss']
        val_loss = losses['val']['loss']
        
        print(f"step {iter_num}: train loss {train_loss:.4f}, val loss {val_loss:.4f}")
        print(f"train ppl {losses['train']['ppl']:.2f}, val ppl {losses['val']['ppl']:.2f}")
        
        # Train/Val gap 監控
        gap = val_loss - train_loss
        gap_ratio = gap / train_loss if train_loss > 0 else 0
        print(f"📈 Train/Val Gap: {gap:.4f} ({gap_ratio*100:.2f}%)")
        
        # PreCo 詳細統計
        if model_name == "preco":
            print(f"PRECO 詳細統計:")
            print(f"CE Loss: {losses['train']['ce_loss']:.4f} / {losses['val']['ce_loss']:.4f}")
            print(f"Kalman 統計: mean={losses['train']['kalman_mean']:.4f}, std={losses['train']['kalman_std']:.4f}")
        
        # wandb 記錄
        if use_wandb:
            log_data = {
                "train/loss": train_loss, "val/loss": val_loss,
                "train/ppl": losses['train']['ppl'], "val/ppl": losses['val']['ppl'],
                "learning_rate": lr, "iter": iter_num,
            }
            
            if model_name == "preco":
                log_data.update({
                    "preco/adaptive_ttt_lr": "native",
                    "preco/kalman_mean": losses['train']['kalman_mean'],
                    "preco/kalman_std": losses['train']['kalman_std'],
                })
            
            wandb.log(log_data)
        
        # 保存最佳檢查點
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            print(f"✅ 新的最佳驗證損失: {best_val_loss:.4f}")
    
    # ========================================
    # 3. 前向和反向傳播
    # ========================================
    for micro_step in tqdm(range(gradient_accumulation_steps)):
        with ctx:
            if model_name == "ttt":
                outputs = model(input_ids=x, labels=y)
                if isinstance(outputs, tuple):
                    logits, loss = outputs[0], outputs[1]
                else:
                    logits, loss = outputs.logits, outputs.loss
                loss = loss / gradient_accumulation_steps
                
            elif model_name == "preco":
                logits, loss_dict = model(x, y, compute_branch_loss=False)
                loss = loss_dict['total_loss'] / gradient_accumulation_steps
                
            else:
                logits, loss = model(x, y)
                loss = loss / gradient_accumulation_steps
        
        # 異步預取下一個批次
        x, y = get_batch('train')
        scaler.scale(loss).backward()
    
    # ========================================
    # 4. 梯度裁剪和優化器步驟
    # ========================================
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad(set_to_none=True)
    
    # ========================================
    # 5. 日誌記錄
    # ========================================
    t1 = time.time()
    dt = t1 - t0
    t0 = t1
    
    if iter_num % (log_interval * 2) == 0:
        lossf = loss.item() * gradient_accumulation_steps
        print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms")
        
        if model_name == "preco":
            print(f"TTT 自適應學習率: 原生機制")
    
    iter_num += 1
    
    # ========================================
    # 6. 終止條件
    # ========================================
    if iter_num > max_iters:
        break

# 清理
if use_wandb:
    wandb.finish()

print("訓練完成！")
print(f"最佳驗證損失: {best_val_loss:.4f}")


\n🚀 開始訓練循環...
總迭代次數: 9600
評估間隔: 200
每次迭代處理 tokens: 32,768
step 0: train loss 10.9541, val loss 10.9543
train ppl 57189.97, val ppl 57201.75
📈 Train/Val Gap: 0.0002 (0.00%)
✅ 新的最佳驗證損失: 10.9543


100%|██████████| 4/4 [00:16<00:00,  4.02s/it]


iter 0: loss 10.9652, time 108140.81ms


100%|██████████| 4/4 [00:16<00:00,  4.02s/it]


iter 20: loss 10.6862, time 16099.57ms


...

iter 9580: loss 5.5412, time 16119.12ms


100%|██████████| 4/4 [00:15<00:00,  3.95s/it]


step 9600: train loss 5.5777, val loss 5.5689
train ppl 264.47, val ppl 262.13
📈 Train/Val Gap: -0.0089 (-0.16%)


100%|██████████| 4/4 [00:15<00:00,  3.95s/it]

iter 9600: loss 5.7583, time 106191.17ms
訓練完成！
最佳驗證損失: 5.4677
